In [7]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

using Profile
using PProf
using ProfileSVG

import Base: show

In [8]:
function kalman_filter_graph()
    x_prev_add = AdditionNode()
    
    x_prior = datavar(:x_prior, Normal{Float64})
    add_1   = constvar(:add_1, 1.0)

    connect!(x_prev_add, :in1, x_prior, 1)
    connect!(x_prev_add, :in2, add_1, 1)
    
    noise = constvar(:noise, Normal(0.0, sqrt(200.0)))
    
    add_x_and_noise = AdditionNode()
    
    x = simplerandomvar(:x)
    
    connect!(x_prev_add, :out, x, 1)
    connect!(add_x_and_noise, :in1, x, 2)
    connect!(add_x_and_noise, :in2, noise, 1)
    
    y = datavar(:y, Float64)
    
    connect!(add_x_and_noise, :out, y, 1)
    
    activate!(x_prev_add)
    activate!(add_x_and_noise)
    
    return x_prior, x, y
end

function kalman(data)
    N = length(data)
    
    x_prior, x, y = kalman_filter_graph()

    link(x_prior, getbelief(x))
    
    update!(x_prior, Normal(0.0, sqrt(10000.0)))
    
    beliefs = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getbelief(x) |> enumerate(), (t) -> beliefs[t[1]] = getdata(t[2]))
    
    for d in data
        update!(y, d)
    end
    
    return beliefs
end

kalman (generic function with 1 method)

In [9]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [12]:
@btime kalman($data)

  950.031 μs (14334 allocations: 639.28 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=6.421866707964923, σ=14.002800840280099)
 Normal{Float64}(μ=-0.5772833890791447, σ=9.950371902099892)
 Normal{Float64}(μ=4.1921649267638035, σ=8.137884587711596)
 Normal{Float64}(μ=0.9935877643644748, σ=7.053456158585984)
 Normal{Float64}(μ=1.0131164886050033, σ=6.311944030978033)
 Normal{Float64}(μ=2.2904204069328733, σ=5.763904177042351)
 Normal{Float64}(μ=5.672203722783613, σ=5.337605126836238)
 Normal{Float64}(μ=6.272716864346323, σ=4.993761694389224)
 Normal{Float64}(μ=5.267376779468818, σ=4.708816093480111)
 Normal{Float64}(μ=7.177504103984075, σ=4.467670516087703)
 Normal{Float64}(μ=6.657955856768954, σ=4.2601432284230505)
 Normal{Float64}(μ=9.342987991483438, σ=4.079085082240021)
 Normal{Float64}(μ=9.699021875090098, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=588.9493933651222, σ=0.582706653680032)
 Normal{Float64}(μ=589.9264905173286, σ=0.5822126418123303)
 Normal{Float64}(μ=590.8969816985898, σ=0.5817198842703781)
 Norm

In [11]:
@time kalman(data)

  0.001487 seconds (14.33 k allocations: 639.281 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=6.421866707964923, σ=14.002800840280099)
 Normal{Float64}(μ=-0.5772833890791447, σ=9.950371902099892)
 Normal{Float64}(μ=4.1921649267638035, σ=8.137884587711596)
 Normal{Float64}(μ=0.9935877643644748, σ=7.053456158585984)
 Normal{Float64}(μ=1.0131164886050033, σ=6.311944030978033)
 Normal{Float64}(μ=2.2904204069328733, σ=5.763904177042351)
 Normal{Float64}(μ=5.672203722783613, σ=5.337605126836238)
 Normal{Float64}(μ=6.272716864346323, σ=4.993761694389224)
 Normal{Float64}(μ=5.267376779468818, σ=4.708816093480111)
 Normal{Float64}(μ=7.177504103984075, σ=4.467670516087703)
 Normal{Float64}(μ=6.657955856768954, σ=4.2601432284230505)
 Normal{Float64}(μ=9.342987991483438, σ=4.079085082240021)
 Normal{Float64}(μ=9.699021875090098, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=588.9493933651222, σ=0.582706653680032)
 Normal{Float64}(μ=589.9264905173286, σ=0.5822126418123303)
 Normal{Float64}(μ=590.8969816985898, σ=0.5817198842703781)
 Norm

In [10]:
@time kalman(data)

  3.571878 seconds (10.72 M allocations: 556.505 MiB, 3.07% gc time)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=6.421866707964923, σ=14.002800840280099)
 Normal{Float64}(μ=-0.5772833890791447, σ=9.950371902099892)
 Normal{Float64}(μ=4.1921649267638035, σ=8.137884587711596)
 Normal{Float64}(μ=0.9935877643644748, σ=7.053456158585984)
 Normal{Float64}(μ=1.0131164886050033, σ=6.311944030978033)
 Normal{Float64}(μ=2.2904204069328733, σ=5.763904177042351)
 Normal{Float64}(μ=5.672203722783613, σ=5.337605126836238)
 Normal{Float64}(μ=6.272716864346323, σ=4.993761694389224)
 Normal{Float64}(μ=5.267376779468818, σ=4.708816093480111)
 Normal{Float64}(μ=7.177504103984075, σ=4.467670516087703)
 Normal{Float64}(μ=6.657955856768954, σ=4.2601432284230505)
 Normal{Float64}(μ=9.342987991483438, σ=4.079085082240021)
 Normal{Float64}(μ=9.699021875090098, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=588.9493933651222, σ=0.582706653680032)
 Normal{Float64}(μ=589.9264905173286, σ=0.5822126418123303)
 Normal{Float64}(μ=590.8969816985898, σ=0.5817198842703781)
 Norm